In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from ipynb.fs.full.Models import *

import time as timer

model = SIR

In [2]:
#time dependencies
times=['lock','exp','fermi','rational']
time=times[0]

In [3]:
#weights weights configurations read from best_params file
fname = 'init_params.csv'
df = pd.read_csv(fname, sep=';')
df.index = df.Model
a_ws = np.array(df.loc[time].a)
r_ws = np.array(df.loc[time].r)
weights = [(float(a_ws[i]),float(r_ws[i])) for i in range(len(a_ws))]

#train
t1 = timer.time()
for time in times:
    models = []
    
    #load models
    for i,country in enumerate(countries):
        models.append(Model(country,model,time=time,start=starts[i]))
        print('')
        
    #fit'n'save for each weight
    for a_w,r_w in weights:
        init_params = getInitParams(model=time, a=a_w, r=r_w)
        
        for i, mod in enumerate(models):
            n_fit=0
            mod.fit(init_params.iloc[i], a=a_w, r=r_w)
            
            if not mod.success: print('begin TRYHARD')
            else: print('Easy SUCCESS')
                
            while (not mod.success) & (n_fit <=10):
                mod.fit(mod.optimal, a=a_w, r=r_w, printer=False)
                n_fit+=1
            if n_fit !=0: print('# add. Fits until SUCCESS:', n_fit)
        best_params = printOptimized(models, params = init_params.columns.tolist())
        fname='params/'+time+'_'+str(a_w)+'_'+str(r_w)+'.csv'
        best_params.to_csv(fname)
t2=timer.time()

Creating SIR for Italy
Loaded country's population
Loaded time series of 116 days
Lockdown at 2020-03-11 -> at day 49
VA-lockdown index: 32

Creating SIR for Germany
Loaded country's population
Loaded time series of 111 days
Lockdown at 2020-03-20 -> at day 58
VA-lockdown index: 41

Creating SIR for Spain
Loaded country's population
Loaded time series of 113 days
Lockdown at 2020-03-14 -> at day 52
VA-lockdown index: 35

Creating SIR for United Kingdom
Loaded country's population
Loaded time series of 109 days
Lockdown at 2020-03-23 -> at day 61
VA-lockdown index: 44

Creating SIR for Belgium
Loaded country's population
Loaded time series of 107 days
Lockdown at 2020-03-17 -> at day 55
VA-lockdown index: 38

Creating SIR for Portugal
Loaded country's population
Loaded time series of 106 days
Lockdown at 2020-03-18 -> at day 56
VA-lockdown index: 39

Start fitting
End fitting
Results:
 final_simplex: (array([[5.41415722e-01, 2.85227187e-01, 7.65548452e-04, 5.91606929e+00,
        2.7947

KeyboardInterrupt: 

In [ ]:
print(f'TOTAL RUNTIME:\nseconds={t2-t1}, minutes={np.round((t2-t1)/60.,2)}, hours={np.round((t2-t1)/3600.,2)}')

# Compact save (still requires previous csv files)

In [ ]:
p_disease=['beta', 'mu', 'I0'] #SIR
#p_disease=['beta_I', 'beta_A', 'mu_I', 'mu_A', 'nu', 'I0']

In [ ]:
mylist=[]
for time in times:
    for a_w,r_w in weights:
        fname = 'params/'+time+'_'+str(a_w)+'_'+str(r_w)+'.csv'
        df = pd.read_csv(fname, index_col=0)

        #for country in df.index:
        for country in countries:
            #load AIC and cost
            AIC=df.loc[country]['AIC']
            cost=df.loc[country]['cost']
            
            #load time params
            p_time = list(df.columns[len(p_disease):-2])
            [df[p][country] for p in p_time]
            t_params = [None]*3
            if time =='lock': t_params = [None] + [df[p][country] for p in p_time]
            elif time in ['exp', 'rational']: t_params = [df[p][country] for p in p_time] + [None]*2
            elif time == 'fermi': t_params = [df[p][country] for p in p_time] + [None]
            else: print('unknown time dep')
            
            #load model characteristics(country, time, weights)
            row = [AIC, cost]+[country, time, a_w, r_w] + [df.loc[country][p] for p in p_disease] + t_params
            mylist.append(row)

cols = ['AIC','cost','country','time','a','r','beta', 'mu', 'I0','alpha','t_lock','beta_lock']
SIRs = pd.DataFrame(mylist,columns=cols).sort_values('AIC')
SIRs.index=range(len(SIRs))
SIRs

In [ ]:
fname='params/SIR_models.csv'
SIRs.to_csv(fname)